### Problem 3

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('constituents.csv')
data
print("Shape of data: ", data.shape)
symbol_list = data.Symbol.tolist()

Shape of data:  (505, 3)


In [4]:
import pandas_datareader.data as web
import yfinance as yf
yf.pdr_override()

start_date = '2012-06-11'
end_date = '2022-06-09'
df = web.get_data_yahoo(symbol_list, start=start_date, end = end_date)

[*********************100%***********************]  505 of 505 completed

14 Failed downloads:
- FB: No data found, symbol may be delisted
- WLTW: No data found, symbol may be delisted
- KSU: No data found, symbol may be delisted
- CERN: No data found, symbol may be delisted
- DISCA: No data found, symbol may be delisted
- VIAC: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- XLNX: No data found, symbol may be delisted
- ANTM: No data found, symbol may be delisted
- PBCT: No data found, symbol may be delisted
- INFO: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- DISCK: No data found, symbol may be delisted
- BLL: No data found, symbol may be delisted


In [5]:
adj_close_data = df['Adj Close']
print("Shape of Data: ", adj_close_data.shape)

Shape of Data:  (2516, 505)


In [6]:
limit = 0.05
drop_list = []
for column in adj_close_data.columns:
  if adj_close_data[column].isnull().mean() > limit:
    drop_list.append(column)

adj_close_data.drop(drop_list, axis=1, inplace=True)
print(adj_close_data.shape)

(2516, 453)


In [7]:
adj_close_data.dropna(axis=0, inplace=True)
print(adj_close_data.shape)

(2429, 453)


In [8]:
def calculate_returns(data, start, end):
  data = data[(data.index <= end) & (data.index >= start)]
  data = data.pct_change().fillna(0)
  returns = (data+1).cumprod()[-1] - 1
  return returns

In [9]:
momentum_dict = {}
start, end = '2021-06-08', '2022-05-08'
for column in adj_close_data.columns:
  data = adj_close_data[column]
  momentum = calculate_returns(data, start, end)
  momentum_dict[column] = momentum

In [10]:
momentum_df = pd.DataFrame(momentum_dict.values(), index=momentum_dict.keys(), columns=['12-1 Momentum'])

In [11]:
momentum_df['q'] = pd.qcut(momentum_df['12-1 Momentum'],5,labels=[1,2,3,4,5]).astype(int)
momentum_df.head()

,12-1 Momentum,q
A,-0.122589,2
AAL,-0.263419,1
AAP,0.047392,4
AAPL,0.248056,5
ABC,0.363049,5


In [12]:
def return_signal(q, buy_signal=5, sell_signal=1):
  if q  >= buy_signal:
    return 1
  if q <= sell_signal:
    return -1
  return 0
momentum_df['signal'] = momentum_df['q'].apply(return_signal)
momentum_df.head()

,12-1 Momentum,q,signal
A,-0.122589,2,0
AAL,-0.263419,1,-1
AAP,0.047392,4,0
AAPL,0.248056,5,1
ABC,0.363049,5,1


In [13]:
momentum_df['weight'] = 1/(momentum_df['signal'] == 1).sum()  * momentum_df['signal']
momentum_df.head()

,12-1 Momentum,q,signal,weight
A,-0.122589,2,0,0.000000
AAL,-0.263419,1,-1,-0.010989
AAP,0.047392,4,0,0.000000
AAPL,0.248056,5,1,0.010989
ABC,0.363049,5,1,0.010989


### Problem 4

####(a)

In [14]:
class Solution:
    def reverseStr(self, s: str, k: int) -> str:
        return_string = ""
        str_length = len(s)
        start = 0
        while str_length >= start + 2 * k:
            return_string += s[start:start+k][::-1]
            return_string += s[start+k:start+2*k]
            start += 2 * k
        if k <= str_length - start < 2* k:
            return_string += s[start: start+k][::-1]
            return_string += s[start+k:]
        else:
            return_string += s[start:][::-1]
        return return_string

####(b)

In [15]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution:
    def hasCycle(self, head) -> bool:
        if head is None:
            return False
        components = []
        cnt = head
        while cnt:
            if cnt in components:
                return True
            components.append(cnt)
            cnt = cnt.next
        return False

#### (c)

In [16]:
class Solution:
  def mergeLists(head1, head2):
    merged_list = SinglyLinkedList()
    while head1 or head2:
        if (head1 and head2 and head1.data < head2.data):
            merged_list.insert_node(head1.data)
            head1 = head1.next
        elif (head1 and head2 and head2.data <= head1.data):
            merged_list.insert_node(head2.data)
            head2 = head2.next
        elif head1:
            merged_list.insert_node(head1.data)
            head1 = head1.next
        elif head2:
            merged_list.insert_node(head2.data)
            head2 = head2.next  
   
    return merged_list.head